In [1]:
import pandas as pd

data = pd.read_csv("https://raw.githubusercontent.com/4GeeksAcademy/linear-regression-project-tutorial-pn/refs/heads/main/medical_insurance_cost.csv")
data.head(10)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
5,31,female,25.740,0,no,southeast,3756.62160
6,46,female,33.440,1,no,southeast,8240.58960
7,37,female,27.740,3,no,northwest,7281.50560
8,37,male,29.830,2,no,northeast,6406.41070
9,60,female,25.840,0,no,northwest,28923.13692


In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [3]:
data = data.drop_duplicates().reset_index(drop = True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1337 entries, 0 to 1336
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1337 non-null   int64  
 1   sex       1337 non-null   object 
 2   bmi       1337 non-null   float64
 3   children  1337 non-null   int64  
 4   smoker    1337 non-null   object 
 5   region    1337 non-null   object 
 6   charges   1337 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.2+ KB


In [6]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder



categorical_columns = data.select_dtypes(include=['object']).columns.tolist()
encoder = OneHotEncoder(sparse_output=False)

one_hot_encoded = encoder.fit_transform(data[categorical_columns])
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

df_encoded = pd.concat([data, one_hot_df], axis=1)
df_encoded = df_encoded.drop(categorical_columns, axis=1)
df_encoded = df_encoded.drop("sex_male", axis=1)
df_encoded = df_encoded.drop("smoker_no", axis=1)

df_encoded



# data["sex_n"] = pd.factorize(data["sex"])[0]
# data["smoker_n"] = pd.factorize(data["smoker"])[0]
# data["region_n"] = pd.factorize(data["region"])[0]
# num_variables = ["age", "bmi", "children", "sex_n", "smoker_n", "region_n", "charges"]

# scaler = MinMaxScaler()
# scal_features = scaler.fit_transform(data[num_variables])
# df_encoded = pd.DataFrame(scal_features, index = data.index, columns = num_variables)
# df_encoded.head()


,age,bmi,children,charges,sex_n,smoker_n,region_n,sex_female,smoker_yes,region_northeast,region_northwest,region_southeast,region_southwest
0,19,27.900,0,16884.92400,0,0,0,1.0,1.0,0.0,0.0,0.0,1.0
1,18,33.770,1,1725.55230,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0
2,28,33.000,3,4449.46200,1,1,1,0.0,0.0,0.0,0.0,1.0,0.0
3,33,22.705,0,21984.47061,1,1,2,0.0,0.0,0.0,1.0,0.0,0.0
4,32,28.880,0,3866.85520,1,1,2,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,50,30.970,3,10600.54830,1,1,2,0.0,0.0,0.0,1.0,0.0,0.0
1333,18,31.920,0,2205.98080,0,1,3,1.0,0.0,1.0,0.0,0.0,0.0
1334,18,36.850,0,1629.83350,0,1,1,1.0,0.0,0.0,0.0,1.0,0.0
1335,21,25.800,0,2007.94500,0,1,0,1.0,0.0,0.0,0.0,0.0,1.0


In [7]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest, f_regression

y = df_encoded["charges"]
X = df_encoded.drop("charges", axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 42)
selection_model = SelectKBest(f_regression, k = 4)
selection_model.fit(X_train, y_train)

selected_columns = X_train.columns[selection_model.get_support()]
X_train_sel = pd.DataFrame(selection_model.transform(X_train), columns = selected_columns)
X_test_sel = pd.DataFrame(selection_model.transform(X_test), columns = selected_columns)

scaler = preprocessing.StandardScaler().fit(X_train_sel, y_train)
X_train_sel = scaler.fit_transform(X_train_sel)
X_test_sel = scaler.transform(X_test_sel)
X_train_sel

array([[-0.44225244,  1.37404961,  0.50570854, -0.50570854],
       [-1.50901428,  0.127246  ,  0.50570854, -0.50570854],
       [ 0.34003957,  1.38230659, -1.9774236 ,  1.9774236 ],
       ...,
       [-1.36677937, -1.41680881,  0.50570854, -0.50570854],
       [-0.08666516, -0.4259715 ,  0.50570854, -0.50570854],
       [-0.30001753,  0.87202537,  0.50570854, -0.50570854]],
      shape=(1203, 4))

In [8]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train_sel, y_train)

LinearRegression()

In [9]:
y_pred = model.predict(X_test_sel)
y_pred

array([ 8384.98679722,  4216.55020264, 13301.81580432, 31109.34088593,
        8790.60781816, 13144.11366451, 29773.94668845,  2002.31015463,
       11006.34432952, 10344.81396664, 11021.25302509, 34283.08759485,
       31125.5092245 , 18292.44985934,  9726.49179272,  9318.47643042,
        4248.76846962, 32653.30861136,  2841.39294915,  5482.90006596,
        4248.76846962, 30376.10052381, 14784.97301122, 31235.51378534,
       31985.91080529,  4706.09628439, 35838.43497616, 38068.66248025,
       11008.14008552, 12288.47279749,  6465.79259289, 13261.84389688,
         661.76550946, 12540.90903757, 41161.33067597, 12608.33849821,
        2594.91459352,  2680.72585158, 32490.18470011,  9369.55666999,
        6923.30129016, 29645.04565161, 34699.33677276, 12005.31475377,
        6798.77123317,  4022.64854991,  6044.54588087,  9184.57785947,
        4065.49497386,  8929.4759543 ,  6307.79116042, 12203.26012477,
       31303.36094882,  4353.5041726 , 10799.74744007,  9692.68662114,
      

In [10]:
from sklearn.metrics import mean_squared_error, r2_score

print(f"Mean squared error: {mean_squared_error(y_test, y_pred)}")
print(f"Coefficient of determination: {r2_score(y_test, y_pred)}")

Mean squared error: 38278614.02100202
Coefficient of determination: 0.7867357462850342
